<a href="https://colab.research.google.com/github/sras202/Network-with-three-inhibitory-cell-types/blob/master/Repeated_K_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving train.csv to train (2).csv


In [ ]:
import io

train_data=pd.read_csv(io.BytesIO(uploaded['train.csv']))

train_data.head()


,SMILES,SENTENCE
0,C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose"
1,COC(=O)OC,"fresh,ethereal,fruity"
2,Cc1cc2c([nH]1)cccc2,"resinous,animalic"
3,C1CCCCCCCC(=O)CCCCCCC1,"powdery,musk,animalic"
4,CC(CC(=O)OC1CC2C(C1(C)CC2)(C)C)C,"coniferous,camphor,fruity"


In [ ]:
df_train=pd.DataFrame(train_data)

df_train_arr=df_train.to_numpy()

df_train_des=df_train_arr[:,1]

des_list=[]

for i in range(0,len(df_train_des)):
    des_list.append(df_train_des[i].split(','))
    
des_list_new=[]

for i in range(0,len(des_list)):
    for j in range(0,len(des_list[i])):
        des_list_new.append(des_list[i][j])

des_list_new_arr=np.unique(des_list_new)

mlb=MultiLabelBinarizer()

one_hot_labels=mlb.fit_transform(des_list)

one_hot_classes=mlb.classes_


In [ ]:
x=np.load(r'mol2vec_features.npy')

print(np.shape(x))

(4316, 300)


In [ ]:
# Get the model

def get_model(n_inputs,n_outputs):
  model=Sequential()
  model.add(Dense(80,input_dim=n_inputs,kernel_initializer='he_uniform',activation='relu'))
  model.add(Dense(n_outputs,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam')
  return model

In [ ]:
# Evaluate the model using repeated k-fold cross valiadation

def evaluate_model(X,y):
    results=list()
    n_inputs,n_outputs=X.shape[1],y.shape[1]

    # Define evaluation procedure

    cv=RepeatedKFold(n_splits=10,n_repeats=8,random_state=1)

    y_val=[]
    y_pred=[]

    js=[]

    # enumerate folds

    for train_ix,test_ix in cv.split(X):
        # prepare data
        X_train,X_test=X[train_ix],X[test_ix]
        y_train,y_test=y[train_ix],y[test_ix]

        # define model

        model=get_model(n_inputs,n_outputs)

        # Fit model

        model.fit(X_train,y_train,verbose=0,epochs=100)

        # Make a prediction on test set

        yhat=model.predict(X_test)

        # Round probabilities to class labels

        yhat=yhat.round()

        y_val.append(y_test)
        y_pred.append(yhat)
        # Calculate accuracy 

        acc=accuracy_score(y_test,yhat)

        js=jaccard_score(y_test,yhat,average='samples')

        # Store result

        print('>%.3f'%acc)

        print('Jaccard_score :',js)

        results.append(acc)

    return results,y_val,y_pred




In [ ]:
X,y=x,one_hot_labels

In [ ]:
# evaluate model
results,y_val,y_pred=evaluate_model(X,y)


print('Accuracy : %.3f (%.3f)'%(mean(results),std(results)))

>0.016
Jaccard_score : 0.1279320987654321
>0.025
Jaccard_score : 0.1201058201058201
>0.039
Jaccard_score : 0.12485119047619048
>0.030
Jaccard_score : 0.1355875220458554
>0.016
Jaccard_score : 0.12786596119929453
>0.039
Jaccard_score : 0.12294606114050557
>0.016
Jaccard_score : 0.12390067395867861
>0.021
Jaccard_score : 0.12341177770412107
>0.021
Jaccard_score : 0.1420101746667872
>0.023
Jaccard_score : 0.13247155010496078
>0.032
Jaccard_score : 0.14266148589065256
>0.028
Jaccard_score : 0.1359595458553792
>0.035
Jaccard_score : 0.14419091710758375
>0.021
Jaccard_score : 0.14309413580246913
>0.023
Jaccard_score : 0.13590351263962375
>0.025
Jaccard_score : 0.1249779541446208
>0.028
Jaccard_score : 0.14132140095017123
>0.026
Jaccard_score : 0.13999926343313815
>0.030
Jaccard_score : 0.1294055905424815
>0.039
Jaccard_score : 0.14588719478510662
>0.012
Jaccard_score : 0.13125734861845972
>0.028
Jaccard_score : 0.12170506319811875
>0.023
Jaccard_score : 0.1452491181657848
>0.037
Jaccard_scor

In [ ]:
print(y_val)

[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]]), array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
   

In [ ]:
def jaccard(a,b):
    return len(a.intersection(b))/len(a.union(b))

In [ ]:
print(y_pred)

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0

In [ ]:
print(len(results))

30


In [ ]:
print(np.shape(y_val[0]))

(432, 109)


(30,)

In [ ]:
y_val_s=set(y_val)

TypeError: ignored